In [116]:
from models import *
from train import*
import numpy as np
#from view_atoms_mgmno import view_atoms_classifier
import numpy as np
from ase import Atoms
from ase.io import read,write
import sys
import torch

In [117]:

def view_atoms_classifier(image,mg_index,mn_index, o_index, view=True):
	x= image.reshape(-1,3)
	mg = x[2:10,:]
	mn = x[10:18,:]
	o = x[18:,:]

	l = x[0,:]*30
	a = x[1,:]*180
	c = np.hstack((l,a))
	atoms = Atoms('H')
	atoms.set_cell(c)
	cell = atoms.get_cell()
	t = np.isnan(cell)
	tt = np.sum(t)
	isnan = False
	if not tt == 0:
		isnan = True
		print(cell)
		print(l)
		print(a)
	#_,mg_index = torch.max(mg_label,dim=1)
	#_,mn_index = torch.max(mn_label,dim=1)
	#_,o_index = torch.max(o_label,dim=1)
	
	#mg_index = mg_index.reshape(8,).detach().cpu().numpy()
	#mn_index = mn_index.reshape(8,).detach().cpu().numpy()
	#o_index = o_index.reshape(12,).detach().cpu().numpy()
	
	
	mg_pos = mg[np.where(mg_index)]
	mn_pos = mn[np.where(mn_index)]
	o_pos = o[np.where(o_index)]
	
	n_mg = len(mg_pos)
	n_mn = len(mn_pos)
	n_o = len(o_pos)
	
	if n_mg == 0:
		mg_pos = np.array([0.1667,0.1667,0.1667]).reshape(1,3)
		n_mg = 1
	if n_mn == 0:
		mn_pos = np.array([0.1667,0.1667,0.1667]).reshape(1,3)
		n_mn = 1
		
	if n_o == 0:
		o_pos = np.array([0.1667,0.1667,0.1667]).reshape(1,3)
		n_o = 1

	pos = np.vstack((mg_pos,mn_pos,o_pos))
	scaled_pos = back_to_10_cell(pos,n_mg,n_mn,n_o)
	atoms = back_to_real_cell(scaled_pos, cell, n_mg,n_mn,n_o)
	atoms.set_pbc([1,1,1])
	if view :
		atoms.edit()
			
	return atoms, x, isnan

In [118]:
class Options:
    def __init__(self):
        self.batch_size = 32
        self.latent_dim = 512
        self.input_dim = 512+28+1

opt = Options()

In [119]:
generator = Generator(opt)
discriminator = Discriminator(opt)
net_Q = QHead_(opt)
device = torch.device('cpu')
generator.load_state_dict(torch.load('/Users/oridwan/Desktop/generator_500', 
                                     map_location=device))
discriminator.load_state_dict(torch.load('/Users/oridwan/Desktop/discriminator_500', 
                                     map_location=device))
net_Q.load_state_dict(torch.load('/Users/oridwan/Desktop/Q_500', 
                                     map_location=device))

<All keys matched successfully>

In [161]:
FloatTensor = torch.FloatTensor

fake_c_mg_int = np.ones(opt.batch_size, dtype = int)*2-1 #np.random.randint(0, 1, opt.batch_size) -1
fake_c_mg_int = torch.tensor(fake_c_mg_int).type(LongTensor)

print("fake_c_mg_int",fake_c_mg_int)
fake_c_mn_int = np.ones(opt.batch_size,dtype = int)*2 -1 #np.random.randint(, 1, opt.batch_size) -1
print("fake_c_mn_int",fake_c_mn_int)
fake_c_o_int = np.ones(opt.batch_size, dtype = int)*6 -1 #np.random.randint(2,2,opt.batch_size)
print("fake_c_o_int",fake_c_o_int)
natoms_fake = fake_c_mg_int + fake_c_mn_int + fake_c_o_int +3
natoms_fake = natoms_fake.float()
print('natoms_fake_before',natoms_fake)
natoms_fake = Variable(FloatTensor(natoms_fake)/(28.0)).unsqueeze(-1)
print('natoms_fake_after',natoms_fake)

z = FloatTensor(np.random.normal(0,1,(opt.batch_size, opt.latent_dim)))


fake_c_mg_int tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
fake_c_mn_int [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
fake_c_o_int [5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5]
natoms_fake_before tensor([10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10.])
natoms_fake_after tensor([[0.3571],
        [0.3571],
        [0.3571],
        [0.3571],
        [0.3571],
        [0.3571],
        [0.3571],
        [0.3571],
        [0.3571],
        [0.3571],
        [0.3571],
        [0.3571],
        [0.3571],
        [0.3571],
        [0.3571],
        [0.3571],
        [0.3571],
        [0.3571],
        [0.3571],
        [0.3571],
        [0.3571],
        [0.3571],
        [0.3571],
        [0.3571],
        [0.3571],
        [0.3571],
        [0.3571],
      

In [162]:
fake_c_mg = to_categorical(fake_c_mg_int,num_columns = 8)
print("fake_c_mg",fake_c_mg)
fake_c_mn = to_categorical(fake_c_mn_int,num_columns = 8)
print("fake_c_mn",fake_c_mn)
fake_c_o = to_categorical(fake_c_o_int, num_columns = 12)
print("fake_c_o",fake_c_o)


fake_c_mg tensor([[0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0.,

In [163]:
fake = generator(z,fake_c_mg,fake_c_mn,fake_c_o,natoms_fake)
print("++++++++++++++++++++++++++",fake.shape)
#fake_feature, G = discriminator(fake)
fake_mg_label, fake_mn_label, fake_o_label, fake_mg_cat, fake_mn_cat, fake_o_cat, fake_cell_pred = net_Q(fake)

print(fake[0])

gen_input torch.Size([32, 541])
l1 layer in generator torch.Size([32, 3584])
Convert it to pixel data torch.Size([32, 128, 28, 1])
Conv1 layer in generator torch.Size([32, 256, 28, 3])
Conv2 layer in generator torch.Size([32, 512, 28, 3])
Conv3 layer in generator torch.Size([32, 256, 28, 3])
Conv4 layer in generator torch.Size([32, 1, 28, 3])
flatten Conv4 layer in generator torch.Size([32, 84])
Apply activation to get pos torch.Size([32, 1, 28, 3])
Get Cell from position torch.Size([32, 6])
Final generator output torch.Size([32, 1, 30, 3])
++++++++++++++++++++++++++ torch.Size([32, 1, 30, 3])
Input of classifier torch.Size([32, 1, 30, 3])
model_cell in classifier torch.Size([32, 64, 2, 1])
model_mg in classifier torch.Size([32, 2, 8, 1])
model_mn in classifier torch.Size([32, 2, 8, 1])
model_o in classifier torch.Size([32, 2, 12, 1])
final output of mg from classifier torch.Size([32, 2, 8, 1])
final output of mn from classifier torch.Size([32, 2, 8, 1])
final output of o from classifi

In [164]:
print(fake_mg_label[0])
print(fake_mn_label[0])
print(fake_o_label[0])

print(fake[0])


tensor([[[9.8419e-07],
         [1.0790e-06],
         [1.2076e-06],
         [1.0000e+00],
         [1.0000e+00],
         [1.0000e+00],
         [1.0000e+00],
         [1.0000e+00]],

        [[1.0000e+00],
         [1.0000e+00],
         [1.0000e+00],
         [1.9848e-06],
         [1.9803e-06],
         [1.9801e-06],
         [1.9782e-06],
         [2.0757e-06]]], grad_fn=<SelectBackward0>)
tensor([[[1.1452e-05],
         [9.9979e-01],
         [2.5280e-05],
         [9.9981e-01],
         [9.9972e-01],
         [9.9981e-01],
         [9.9981e-01],
         [9.9981e-01]],

        [[9.9999e-01],
         [2.0961e-04],
         [9.9997e-01],
         [1.8636e-04],
         [2.7857e-04],
         [1.8642e-04],
         [1.8769e-04],
         [1.8644e-04]]], grad_fn=<SelectBackward0>)
tensor([[[7.3036e-09],
         [9.9932e-01],
         [6.7416e-09],
         [3.3265e-09],
         [6.0215e-09],
         [4.9449e-09],
         [9.0728e-09],
         [9.5546e-01],
         [9.7986e-

In [165]:
_,mg_index = torch.max(fake_mg_label,dim=1)
_,mn_index = torch.max(fake_mn_label,dim=1)
_,o_index = torch.max(fake_o_label,dim=1)
print(mg_index[0,:,0])

print(mn_index[0, :, 0])
print(o_index[0, :, 0])
print(mg_index.shape)
print(fake.shape)
'''
import torch

def create_tensor(batch_size, vector_size, num_ones):
    # Initialize a zero tensor of shape (batch_size, vector_size, 1)
    tensor = torch.zeros(batch_size, vector_size, 1)

    # For each vector in the batch
    for i in range(batch_size):
        # Randomly select `num_ones` distinct positions
        ones_positions = torch.randperm(vector_size)[:num_ones]
        #print(ones_positions)
        # Set the selected positions to 1
        tensor[i, ones_positions] = 1
        #print(ones_positions)

    return tensor


mg_index = create_tensor(32,8,1)
mn_index = create_tensor(32,8,1)
o_index = create_tensor(32,12,3)

print('mg_index',mg_index)
print('mn_index',mn_index)
print('o_index',o_index)
'''
print(fake_mg_cat[0])
print(fake_mn_cat[0])
print(fake_o_cat[0])

tensor([1, 1, 1, 0, 0, 0, 0, 0])
tensor([1, 0, 1, 0, 0, 0, 0, 0])
tensor([1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0])
torch.Size([32, 8, 1])
torch.Size([32, 1, 30, 3])
tensor([2.8347e-11, 3.0912e-13, 1.0000e+00, 6.5398e-20, 6.6857e-10, 3.5859e-16,
        2.2670e-21, 1.2664e-11], grad_fn=<SelectBackward0>)
tensor([1.5556e-03, 9.2717e-02, 5.5451e-04, 1.1580e-02, 3.3880e-02, 8.5896e-01,
        7.1019e-04, 4.0586e-05], grad_fn=<SelectBackward0>)
tensor([9.5593e-12, 1.4743e-05, 3.4165e-05, 1.0472e-29, 5.7292e-17, 6.0792e-21,
        6.8171e-15, 1.9317e-07, 9.9904e-01, 5.5674e-05, 8.5489e-04, 1.3972e-08],
       grad_fn=<SelectBackward0>)


In [166]:
from ase.visualize import view


for i in range(opt.batch_size):
    atom, _, _ = view_atoms_classifier(fake[i].detach().numpy(),
                                  mg_index[i, :, 0].numpy(),
                                  mn_index[i, :, 0].numpy(),
                                  o_index[i, :, 0].numpy(),
                                  view=False)
    #print(i, mg_index[i])
    atom.write('cif/'+str(i)+'.cif', format='cif')
    print(i, atom)
    #view(atom, viewer='ngl')

0 Atoms(symbols='Mg3Mn2O6', pbc=True, cell=[[4.123617172241211, 0.0, 0.0], [0.5036955444047091, 5.2965686581040226, 0.0], [0.5794107743318679, -0.823806097782632, 4.755911982056253]])
1 Atoms(symbols='Mg2Mn2O6', pbc=True, cell=[[8.171021461486816, 0.0, 0.0], [0.44295228288460453, 4.5921146169125695, 0.0], [-0.35489337535946536, -3.3558574782980113, 3.3468393560540166]])
2 Atoms(symbols='Mg2Mn2O6', pbc=True, cell=[[4.881555557250977, 0.0, 0.0], [1.4225941787155505, 8.593392248454908, 0.0], [-1.1319062813869685, 0.8432789720541919, 5.507864508493293]])
3 Atoms(symbols='Mg3Mn2O6', pbc=True, cell=[[6.36337423324585, 0.0, 0.0], [-0.9870395010844506, 6.590831040396185, 0.0], [1.157805020990594, 1.616967632081531, 2.430286955006509]])
4 Atoms(symbols='Mg4Mn2O5', pbc=True, cell=[[3.9229354858398438, 0.0, 0.0], [-2.429631205386499, 9.430760441351579, 0.0], [-3.0456426903106064, -1.073847368039581, 4.6598510217467]])
5 Atoms(symbols='Mg2Mn2O6', pbc=True, cell=[[10.89255142211914, 0.0, 0.0], [1.7

AssertionError: 

In [149]:
from ase.visualize import view
print(atom)
view(atom, viewer='ngl')


Atoms(symbols='MgMn2O6', pbc=True, cell=[[4.79154634475708, 0.0, 0.0], [3.074703727425063, 7.795187029996072, 0.0], [2.7275274223124844, 3.0255704637452703, 4.422157712094152]])
